In [4]:
# We import all our dependencies.
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.optimizer import Optimizer
import sys,os,shutil
import time
sys.path.append('../')


from models.lvae import LadderVAE
from boilerplate import boilerplate
import lib.utils as utils
import training
import glob
import zipfile
import urllib
from tifffile import imread, imsave
from matplotlib import pyplot as plt
from tqdm import tqdm

from pathlib import Path

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

# data
data_path = Path(r"E:\dl_monalisa\Data\Vim_live_timelapse_Monalisa1_35nm\dump\recon\dyn_time2")
crop_size = 400
im_size = 1060
start = im_size//2-crop_size//2
stop = im_size//2+crop_size//2


# model
model = torch.load("./Trained_model/model/Vim_fixed_SNR0_best_vae.net")
model.mode_pred=True
model.eval()

# define and create saving location
overwrite = True
save_path = Path(r"E:\dl_monalisa\Data\Vim_live_timelapse_Monalisa1_35nm\dump\denoised_UNET_mse_1kepoch_LowtoAvg\dyn_time2")
if os.path.exists(save_path):
    if not overwrite:
        raise Exception("Save dir already exists")
    else:
        print("Overwriting existing directory, pausing for 3sec in case you want to stop.")
        time.sleep(3)
        print("Proceeding")
        shutil.rmtree(save_path)
        os.makedirs(save_path)
else:
    os.makedirs(save_path)



cuda
Overwriting existing directory, pausing for 3sec in case you want to stop.
Proceeding


In [8]:

# prediction
num_samples = 20
list_files = os.listdir(data_path)
for i in range(len(list_files)):
    name_file = list_files[i]
    if name_file.split('.')[-1] not in ['tif','tiff']:
        print(f"Skipping {name_file}")
        continue

    print(f"Processing {name_file}")
    stack = imread(data_path / name_file)[:,start:stop,start:stop]
    stack[stack<0]=0


    # predict
    pred_stack = pred_timelapse = np.empty((stack.shape[0],stack.shape[1],stack.shape[2]))
    for i in range(stack.shape[0]):
        print(f"frame #{i}")
        frame = stack[i]
        img_mmse, _ = boilerplate.predict(frame,num_samples,model,None,device,False)

    # saving
    full_save_path = save_path / name_file

    imsave(full_save_path,pred_stack)

    break

Skipping Thumbs.db
Processing v02.tif
frame #0


 70%|███████   | 14/20 [05:54<02:31, 25.33s/it]


KeyboardInterrupt: 